In [1]:
## !pip3 install -q -r requirements.txt
# !pipreqsnb --force

In [2]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from IPython import display

import gymnasium as gym
import numpy as np
import random

import copy

import torch
import torch.nn as nn
import tqdm.notebook as tqdm

from torch.distributions import Normal
from torch.utils.data import Dataset, DataLoader

In [3]:
from agents import ModelFreeAgent, CRandAgent
from aux_func import LinearAR, ExponentialAR, AnnealingRate

In [4]:
ENV = gym.make("Pendulum-v1", render_mode="rgb_array")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# Pendulum-v1LunarLanderContinuous-v2

In [5]:
class PPOIDataset(Dataset):
    '''intermediate structure for PyTorch DataLoader'''
    def __init__(self, states, actions, returns, advantage_values):
        self.S = states
        self.A = actions
        self.Af = advantage_values
        self.R = returns
        assert len(self.S) == len(self.A) == len(self.Af) == len(self.R), 'wrong input'

    def __len__(self):
        return len(self.S)

    def __getitem__(self, i):
        return self.S[i], self.A[i], self.Af[i], self.R[i], i

In [27]:
class PPOAgent(ModelFreeAgent, CRandAgent):
    def __init__(self, env, aid_to_str=None, hidden_d=(64, 32), device=DEVICE):
        super().__init__(env=env, aid_to_str=aid_to_str)
        self.d_actions = env.unwrapped.action_space.shape[0]
        self.a_ll = torch.Tensor(env.action_space.low)
        self.a_ul = torch.Tensor(env.action_space.high)
        
        self.loss = nn.MSELoss()
        self.device = device
        self.frozen_pi = None
        self.z_eps = 1E-3 # for internal zero-division issues
        
        # model for a value function (scalar)
        self.V_model = nn.Sequential(
            nn.Linear(in_features=self.d_states, out_features=hidden_d[0]),
            nn.ReLU(),
            nn.Linear(in_features=hidden_d[0], out_features=hidden_d[1]),
            nn.ReLU(),
            nn.Linear(hidden_d[1], 1))
        
        # predicts parameters mu, std of action distribution (implying that's a Normal d.)
        self.pi_model = nn.Sequential(
            nn.Linear(in_features=self.d_states, out_features=hidden_d[0]),
            nn.ReLU(),
            nn.Linear(in_features=hidden_d[0], out_features=hidden_d[1]),
            nn.ReLU(),
            nn.Linear(hidden_d[1], 2 * self.d_actions),
            # nn.Tanh()
        )
        
        # on-the-fly advantage values calculation
        self.fly = self.adv
        self.Af = None
        
    def adv(self, queue_6, terminated, last_state, gamma):
        """Calculates advantage function online, throughout current trajectory,
            each tensor allows backprop gradient flow"""
        if len(queue_6) == 6:  # sarsar
            # cast to torch tensors on device with float dtype (evades longint inheritance)
            s, a, r, sx, ax, rx = map(lambda x: torch.tensor(x, dtype=torch.float, device=self.device), list(queue_6))
            self.Af.append(r + gamma * self.V_model(sx).detach() - self.V_model(s))
            # additional xxx|sar + terminal state as next action doesn't happen
            if terminated: # done or exhausted
                s, a, r, sx = sx, ax, rx, torch.tensor(last_state, dtype=torch.float, device=self.device)
                self.Af.append(r + gamma * self.V_model(sx).detach() - self.V_model(s))
                

    def pi_distr(self, states_tensor):
        """transforms net output of current policy model to parameters of distribution on actions at these states (as multidimensional continuous RV)
        torch-compliant method, allows backprop gradient flow"""
        # split net output on two parts
        mean_raw, std_raw = torch.split(self.pi_model(states_tensor), self.d_actions, dim=-1)
        # ensure that they do obey limits (as parameters of Normal distribution)
        mean = torch.clamp(mean_raw, self.a_ll + self.z_eps ** 2, self.a_ul - self.z_eps ** 2) # clipping for a given action interval
        std = nn.functional.threshold(std_raw, threshold=0, value=self.z_eps) # prevent non-positive std
        if torch.isnan(mean_raw).any():
            print('FUGG')
        return Normal(mean, std)
        
        
    def walk_ga(self, max_length, gamma):
        """transforms standard results after tracing a route, yields
            standard SA as numpy arrays, total reward, then G-returns and Advantage value tensors
            only Af values have .grad attribute"""
        self.Af = []
        trajectory = super().walk(max_length=max_length, gamma=gamma)
        S, A, R = trajectory['s'], trajectory['a'], trajectory['r']
        #get rewards serie (summands, w/ discounting)
        gamma_ = np.cumprod(np.concatenate((np.atleast_1d(1.), np.tile(np.array(gamma), len(R) - 1))))
        R_discounted = gamma_ * np.array(R)
        G = np.array([np.sum(R_discounted[t:]) / gamma ** t for t in range(len(R_discounted))])
        return torch.Tensor(np.array(S)), torch.Tensor(np.array(A)), torch.sum(torch.Tensor(R)).unsqueeze(0), torch.tensor(G, dtype=torch.float), torch.stack(self.Af)
        

    def act(self, state):
        """samples an action from a normal distribution (mean, std are given by NN)"""
        # convert to torch tensor with explicit dtype just in case
        state_tensor = torch.tensor(state, dtype=torch.float, device=self.device)
        # model accepts non-batched input, proceed as is
        action_distr = self.pi_distr(state_tensor)
        return action_distr.sample().numpy()

    
    def fit(self, n_epochs=100, n_trajectories=20, max_length=200, lr=0.001, gamma=0.99, batch_size=128, eps_d=None, verbose=None):
        """
        This algorithm performs PPO learning on a bunch (sized n_trajectories) of trajectories (w/ length <= max_length) for n_iterations
            lr defines learning rate of built-in Adam optimizer
            verbose>0 sets up a period of learning process rendering
        NB: .fit internally uses .act method of child class(this), doesn't inherit parental
        """

        self.V_model.to(self.device)
        self.pi_model.to(self.device)
        V_optimizer = torch.optim.Adam(self.V_model.parameters(), lr=lr)
        pi_optimizer = torch.optim.Adam(self.pi_model.parameters(), lr=lr)
        
        epochs_pbar = tqdm.trange(n_epochs, position=0, leave=True, colour="#a2d2ff")
        dh = display.display(display_id=True)
        # use linear annealing rule default or customize given one
        if eps_d is not None:
            eps_d.n_total = n_epochs
        else:
            eps_d = LinearAR(n_iterations=n_iterations, start=0.2)
        self.log, eps_log, V_loss_log, pi_loss_log = [], [], [], []
        for e in epochs_pbar:
            # obtain epsilon
            eps = eps_d(e)
            # trace several routes in order to deal with stochastic environment
            results = [self.walk_ga(max_length, gamma=gamma) for t in range(n_trajectories)]
            # merge results into a big chunk
            S, A, R_tot, G, Af = map(torch.cat, zip(*results)) # nb! default concatenation dim=0
            a_lp = self.pi_distr(S).log_prob(A).detach()
            # batch processing
            curr_data = PPOIDataset(S, A, G, Af)
            for (S_b, A_b, G_b, Af_b, idx) in DataLoader(curr_data, batch_size, shuffle=True):
                # extract policy model probabilities for given actions
                a_lp_curr = self.pi_distr(S_b).log_prob(A_b)
                # numerically stable approach (probabilities might be too close to zero, underflow)
                ratio = torch.exp(a_lp[idx] - a_lp_curr)
                Af_b = Af_b.unsqueeze(-1).detach() # because Dataloader changes (squeezes) that
                # Af_b = G_b.detach() - self.V_model(S_b)
                # forward pass: maximize this quantity (set up as loss) and minimize difference between returns and model of V
                pi_loss = - torch.mean(torch.minimum(ratio * Af_b, torch.clamp(ratio, 1 - eps, 1 + eps) * Af_b))
                V_loss = self.loss(self.V_model(S_b), G_b.detach())
                # double backward pass
                V_optimizer.zero_grad()
                pi_optimizer.zero_grad()
                V_loss.backward()
                pi_loss.backward()
                V_optimizer.step()
                pi_optimizer.step()

            # logging
            eps_log.append(eps)
            V_loss_log.append(V_loss.detach().item())
            pi_loss_log.append(pi_loss.detach().item())
            self.log.append(torch.mean(R_tot).item())
            epochs_pbar.set_postfix_str(f'curr reward: {self.log[-1]:.1f} | V_loss: {V_loss_log[-1]:.2f} | pi_loss: {pi_loss_log[-1]:.2f}', refresh=True)
            # visualization (plotting starts after at least 1 iteration)
            if verbose and e > 0 and (e + 1) % verbose == 0:
                ax = self.learning_curve(title="Rewards")
                ax2 = ax.twinx()
                ax2.tick_params(axis='y', labelcolor='slateblue')
                # sns.lineplot(eps_log, linewidth=0.5, ax=ax2, label="exploration, ε", color='slateblue')
                ax2 = sns.lineplot(V_loss_log, linewidth=0.5, label='V loss', ax=ax2, color='coral')
                sns.lineplot(pi_loss_log, linewidth=0.5, label='π loss', ax=ax2, color='palevioletred')
                dh.update(plt.gcf())
                plt.close()  # because plt.clf() is spurious
        return self.log[-1]

In [28]:
ppo = PPOAgent(env=ENV)
ppo.fit(n_epochs=40, n_trajectories=20, max_length=200, lr=1e-6, eps_d=AnnealingRate(start=0.2), gamma=0.99, batch_size=64, verbose=1)

  0%|          | 0/40 [00:00<?, ?it/s]

FUGG


ValueError: Expected parameter loc (Tensor of shape (64, 1)) of distribution Normal(loc: torch.Size([64, 1]), scale: torch.Size([64, 1])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
       